# Tests Loss
On this notebook we will exercise the expected input/target for segmentation loss.
```python
n_class = 10
batch_size = 4
img_size = 224
preds = torch.randn(batch_size, num_classes, img_size, img_size)
labels = torch.empty(batch_size, img_size, img_size, dtype=torch.long).random_(n_class)

criterion = nn.CrossEntropyLoss()
loss = criterion(preds, labels)
```
#### References
* https://discuss.pytorch.org/t/the-cost-function-for-semantic-segmentation/27668/2
* https://discuss.pytorch.org/t/multi-class-cross-entropy-loss-function-implementation-in-pytorch/19077/12
* https://github.com/CSAILVision/semantic-segmentation-pytorch

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import models
import losses
print("PyTorch Version: ",torch.__version__)
num_classes = 2
batch_size = 10
img_size = 224

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = 'cpu'
print('Device:', device)
num_gpu = torch.cuda.device_count()
num_gpu = 1
batch_size *= num_gpu
print('Number of GPUs Available:', num_gpu)

PyTorch Version:  1.1.0
Device: cpu
Number of GPUs Available: 1


#### Define expected labels, model output

In [10]:
preds = torch.randn(batch_size, num_classes, img_size, img_size).to(device)
labels = torch.empty(batch_size, img_size, img_size, dtype=torch.long).random_(num_classes).to(device)
print('preds.shape:', preds.shape)
print('labels.shape:', labels.shape)

preds.shape: torch.Size([10, 2, 224, 224])
labels.shape: torch.Size([10, 224, 224])


#### Try Cross-Entropy Loss
If you use this loss you don't need to use the softmax activation

In [11]:
criterion = nn.CrossEntropyLoss()
loss = criterion(preds, labels)
print('Loss:',loss)

Loss: tensor(0.9016)


#### Try Focal Loss

In [13]:
# If there are more than 2 classes the alpha need to be a list
criterion = losses.FocalLoss(gamma=2.0, alpha=0.25)
loss = criterion(preds, labels)
print('Loss:',loss)

Loss: tensor(0.2418)


#### Try on Model

In [7]:
img_reference = torch.randn(batch_size,3,224,224).to(device)
img_test = torch.randn(batch_size,3,224,224).to(device)
change_net = models.ChangeNet(num_classes=num_classes)
if num_gpu > 1:
    change_net = nn.DataParallel(change_net)
change_net = change_net.to(device)

preds = change_net([img_reference, img_test])
print('preds.shape:', preds.shape)
criterion = nn.CrossEntropyLoss()
loss = criterion(preds, labels)
print('Loss:',loss)
# Get the max accross each channel of the image
_, preds_max = torch.max(preds, 1)
print('preds_max.shape:', preds_max.shape)

preds.shape: torch.Size([800, 3, 224, 224])
Loss: tensor(1.3346, device='cuda:0', grad_fn=<NllLoss2DBackward>)
preds_max.shape: torch.Size([800, 224, 224])
